In [1]:
# Importar las bibliotecas necesarias
from anthropic import Anthropic
import requests
from io import BytesIO
import base64
from IPython.display import Image, display
import pandas as pd


In [ ]:
from dotenv import load_dotenv
import os

# Cargar las variables del archivo .env
load_dotenv()

# Leer las variables de entorno
api_key = os.getenv("api_key")

In [30]:
def tags_to_dataframe(text):
    """
    Convierte una cadena de texto con tags separados por comas o saltos de línea en un DataFrame con cada tag en una columna.

    Parameters:
    text (str or list): Cadena de texto con los tags separados por comas o una lista con objetos TextBlock.

    Returns:
    pd.DataFrame: DataFrame con los tags en columnas.
    """
    # Si es una lista, tomar el atributo 'text' del primer elemento
    if isinstance(text, list):
        text = text[0].text

    # Reemplazar saltos de línea por comas y dividir por comas
    clean_text = text.replace('\n', ', ')
    tags = clean_text.split(', ')

    # Crear el DataFrame con los tags
    df = pd.DataFrame([tags], columns=[f'Tag_{i+1}' for i in range(len(tags))])
    return df

def display_image(image_url):
    display(Image(url=image_url))

def analyze_fashion_image(image_url, product_description, api_key):
    # Inicializar el cliente de Anthropic
    client = Anthropic(api_key=api_key)
    
    # Descargar la imagen
    response = requests.get(image_url)
    image_data = BytesIO(response.content)
    
    # Convertir la imagen a base64
    base64_image = base64.b64encode(image_data.read()).decode('utf-8')
    
    # Crear el mensaje para Claude con instrucciones más precisas
    message_content = f"""
    Por favor, analiza la imagen proporcionada y clasifica la ropa siguiendo estrictamente las siguientes categorías. Además, utiliza la descripción del producto proporcionada para enriquecer y hacer más precisa la clasificación. Usa exclusivamente DOS palabras de las opciones proporcionadas para cada categoría. No incluyas explicaciones, comentarios adicionales ni texto fuera del formato especificado.

    Descripción del producto: "{product_description}"

    CATEGORÍAS:
    ESTILO GENERAL: Identifica dos estilos predominantes del conjunto completo y selecciona SOLO DOS palabras de la lista:
    - Casual: Ropa informal y cómoda para el día a día.
    - Elegante: Ropa refinada y sofisticada para ocasiones formales.
    - Deportivo: Diseñada para actividades físicas o de entrenamiento.
    - Streetwear: Estilo urbano influenciado por la cultura juvenil y la moda de calle.
    - Clásico: Prendas atemporales con cortes tradicionales.
    - Minimalista: Estilo limpio, simple y sin elementos complejos.
    - Preppy: Inspirado en la moda universitaria tradicional.
    - Bohemio: Estilo relajado, artístico y desenfadado.
    - Vanguardista: Diseños modernos y experimentales, alejados de lo convencional.
    - Monocromatico: Todo el outfit tiene el mismo color.

    PARTE SUPERIOR: Selecciona SOLO DOS características principales de la prenda superior de la lista:
    - Ajustada: Se ciñe completamente al cuerpo.
    - Oversize: Notablemente holgada y grande.
    - Entallada: Ajustada ligeramente en la cintura o torso, sin ser ceñida.
    - Cuello alto: Cubre completamente el cuello.
    - Escote V: Escote en forma de V.
    - Manga corta: Mangas que llegan por encima del codo.
    - Manga larga: Mangas que llegan hasta la muñeca.
    - Sin mangas: Prenda sin mangas.

    PARTE INFERIOR: Selecciona SOLO DOS características principales de la prenda inferior de la lista:
    - Corte recto: Piernas con caída recta, sin estrecharse.
    - Corte ajustado: Corte ceñido completamente a las piernas.
    - Corte acampanado: Se ensancha hacia los pies.
    - Corte ancho: Holgado, con un estilo relajado y talla grande.
    - Tiro alto: Cintura posicionada por encima del ombligo.
    - Tiro bajo: Cintura posicionada en la cadera.
    - Corte slim: Ajustado, pero sin ser completamente ceñido.
    - Rasgado: Prendas que presentan roturas o desgarros.

    ### Ejemplo de formato de respuesta:
    Casual, Streetwear, Ajustada, Manga larga, Corte recto, Holgado
    """

    # Enviar el mensaje a Claude
    message = client.messages.create(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": message_content},
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",
                            "data": base64_image
                        }
                    }
                ]
            }
        ]
    )
    
    return message.content

# Ejemplo de uso con entrada de descripción
image_url = "https://static.pullandbear.net/assets/public/a636/8c27/f74a4030a671/a9e3e1b2d6c6/07685511407-M/07685511407-M.jpg?ts=1721839385132&w=536&f=auto"
product_description = "Jeans standard con cintura con trabillas, cierre de botón y cremallera, y diseño de cinco bolsillos."

print("Analizando imagen y descripción...")

try:
    # Realizar el análisis
    resultado = analyze_fashion_image(image_url, product_description, api_key)
    df = tags_to_dataframe(resultado)
    print("\nResultado en DataFrame:")
    print(df)
except Exception as e:
    print(f"Error en el análisis: {str(e)}")


Analizando imagen y descripción...

Resultado en DataFrame:
    Tag_1    Tag_2      Tag_3        Tag_4        Tag_5    Tag_6
0  Casual  Clásico  Entallada  Manga corta  Corte recto  Holgado


In [25]:
resultado

[TextBlock(citations=None, text='Streetwear, Casual\nManga larga, Ajustada\nCorte recto, Boyfriend', type='text')]

In [11]:
import pandas as pd
import requests
from io import BytesIO
import base64

def analyze_fashion_image(image_url, product_description, api_key):
    """
    Analiza una imagen y una descripción para obtener dos tags representativos
    de la imagen general, la parte superior y la parte inferior. Devuelve un DataFrame.
    """
    # Inicializar el cliente de Anthropic
    client = Anthropic(api_key=api_key)
    
    # Descargar la imagen
    response = requests.get(image_url)
    image_data = BytesIO(response.content)
    
    # Convertir la imagen a base64
    base64_image = base64.b64encode(image_data.read()).decode('utf-8')
    
    # Crear el mensaje para Claude con instrucciones detalladas
    message_content = f"""
    \n\nHuman: Por favor, analiza la imagen proporcionada y clasifica la ropa siguiendo estrictamente las siguientes categorías.
    Además, utiliza la descripción del producto proporcionada para enriquecer y hacer más precisa la clasificación.
    Usa exclusivamente DOS palabras de las opciones proporcionadas para cada categoría. No incluyas explicaciones, comentarios adicionales ni texto fuera del formato especificado.

    Descripción del producto: "{product_description}"

    CATEGORÍAS:
    ESTILO GENERAL: Identifica dos estilos predominantes del conjunto completo y selecciona SOLO DOS palabras de la lista:
    - Casual: Ropa informal y cómoda para el día a día.
    - Elegante: Ropa refinada y sofisticada para ocasiones formales.
    - Deportivo: Diseñada para actividades físicas o de entrenamiento.
    - Streetwear: Estilo urbano influenciado por la cultura juvenil y la moda de calle.
    - Clásico: Prendas atemporales con cortes tradicionales.
    - Minimalista: Estilo limpio, simple y sin elementos superfluos.
    - Preppy: Inspirado en la moda universitaria tradicional.
    - Bohemio: Estilo relajado, artístico y desenfadado.
    - Vanguardista: Diseños modernos y experimentales, alejados de lo convencional.

    PARTE SUPERIOR: Selecciona SOLO DOS características principales de la prenda superior de la lista:
    - Ajustada: Se ciñe completamente al cuerpo.
    - Oversize: Notablemente holgada y grande.
    - Entallada: Ajustada ligeramente en la cintura o torso, sin ser ceñida.
    - Cuello alto: Cubre completamente el cuello.
    - Escote V: Escote en forma de V.
    - Manga corta: Mangas que llegan por encima del codo.
    - Manga larga: Mangas que llegan hasta la muñeca.
    - Sin mangas: Prenda sin mangas.

    PARTE INFERIOR: Selecciona SOLO DOS características principales de la prenda inferior de la lista:
    - Corte recto: Piernas con caída recta, sin estrecharse.
    - Corte ajustado: Ceñido completamente a las piernas.
    - Corte acampanado: Se ensancha hacia los pies.
    - Boyfriend: Holgado, con un estilo relajado y masculino.
    - Tiro alto: Cintura posicionada por encima del ombligo.
    - Tiro bajo: Cintura posicionada en la cadera.
    - Corte slim: Ajustado, pero sin ser completamente ceñido.

    ### Formato de Respuesta:
    Casual, Streetwear, Ajustada, Manga larga, Corte recto, Boyfriend
    \n\nAssistant:
        """
    
    # Enviar el mensaje a Claude Haiku 3.5
    message = client.completions.create(
        model="claude-3",
        max_tokens_to_sample=1000,
        prompt=message_content
    )
    
    # Extraer la respuesta en texto
    response_text = message.get("completion", "").strip()

    # Validar el formato de la respuesta
    tags = response_text.split(', ')
    if len(tags) != 6:
        raise ValueError("La respuesta no tiene el número esperado de tags. Respuesta obtenida: " + response_text)

    # Crear el DataFrame
    df = pd.DataFrame([{
        'tag_general_1': tags[0],
        'tag_general_2': tags[1],
        'tag_superior_1': tags[2],
        'tag_superior_2': tags[3],
        'tag_inferior_1': tags[4],
        'tag_inferior_2': tags[5]
    }])
    
    return df

# Ejemplo de uso con entrada de descripción
image_url = "https://static.pullandbear.net/assets/public/82ef/24d9/e2ab4c44b64c/ca0c27fdec55/07685506406-A1M/07685506406-A1M.jpg?ts=1719233187238&w=644&f=auto"
product_description = "Jeans slim fit con diseño de cinco bolsillos, cintura con trabillas, cierre de botón y cremallera y confeccionados en algodón ligeramente elástico."

try:
    # Analizar la imagen y obtener el DataFrame
    df_resultado = analyze_fashion_image(image_url, product_description, api_key)
    print("\nResultado en DataFrame:")
    print(df_resultado)
except Exception as e:
    print(f"Error en el análisis: {str(e)}")


Error en el análisis: Error code: 404 - {'type': 'error', 'error': {'type': 'not_found_error', 'message': 'model: claude-3'}}
